## ETL

In [1]:
import pandas as pd
import json
import ast

#### Cleaning 'cast' column

In [2]:
df_credits = pd.read_csv('../datasets/credits.csv', delimiter = ',', encoding = 'utf-8')
df_credits.sort_values(by = 'id').head()

,cast,crew,id
4342,"[{'cast_id': 3, 'character': 'Taisto Olavi Kas...","[{'credit_id': '52fe420dc3a36847f800001f', 'de...",2
12947,"[{'cast_id': 5, 'character': 'Nikander', 'cred...","[{'credit_id': '52fe420dc3a36847f8000077', 'de...",3
17,"[{'cast_id': 42, 'character': 'Ted the Bellhop...","[{'credit_id': '52fe420dc3a36847f800011b', 'de...",5
474,"[{'cast_id': 7, 'character': 'Frank Wyatt', 'c...","[{'credit_id': '52fe420dc3a36847f800023d', 'de...",6
256,"[{'cast_id': 3, 'character': 'Luke Skywalker',...","[{'credit_id': '52fe420dc3a36847f8000437', 'de...",11


Veo que claves tienen los diccionarios que estan anidades en la columna 'cast'

In [7]:
df_credits.cast[0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [12]:
columns = ['cast_id', 'character', 'credit_id', 'gender', 'id', 'name', 'order', 'profile_path','movie_id']

In [13]:
cast_dataframe = pd.DataFrame(columns=columns)

In [14]:
cast_dataframe

,cast_id,character,credit_id,gender,id,name,order,profile_path,movie_id


Para el proceso de extraccion de datos dentro de los diccionarios que contienen las columnas 'cast' y 'crew', creo una funcion que automatice el proceso.

In [65]:
def desanidar_datos(column_df, column_id):
    """ Recibe una columna de un dataframe que tenga diccionarios anidados como valores en formato string y devuelve un datafame nuevo con toda la información extraida """
    # Diccionario donde se van a alojar todos los datos comprimidos que contiene la columna
    main_dictionary = {'movie_id':[]}

    # Creo un ciclo para cada valor de la columna, es decir cada campo (field)
    for index, value in enumerate(column_df):
        if value in ('NaN','nan','none','None','NAN') or type(value) != str:
            continue
        else:
            # Elimino algunos caracteres de cada valor para poder hacer una mejor limpieza
            value = value.replace('[','').replace(']','').replace("\\","").replace('{','').replace("'","").replace('"','')
            # Como cada valor tiene varios diccionarios anidados, con split() divido los diccionarios en una lista
            # para asi poder tenerlos por separados
            listas_texto = value.split('}')

            clean_rows = [] # Lista para almacenar lso diccionarios 'limpios'

            # Al hacer la division de los diccionarios con el metodo split(), nos devuelve que cada diccionario a parti del segundo elemento de la lista
            # contiene ', ' al principio de la cadena. Entonces quito eos caracteres para poder limpiar la cadena y asi obtener un formato de diccionario
            for diccionario in listas_texto:
                # Hay algunos diccionario vienen vacios, por lo tantos los salteo y continuo con el loop
                if len(diccionario) < 10:
                    continue
                else:
                    if diccionario[0] == ',':
                        diccionario = diccionario[2:]
                        clean_rows.append(diccionario)
                    else:
                        clean_rows.append(diccionario)

            blocks = []

            # Una vez que tengo los diccionarios con un mejor formato para su tratamiento, procedo a realizar otra division
            # a partir de las comas (','). Con esta division obtengo en una lista varios 'bloques' que se asemejan
            # mejor a la estructura de un diccionario.
            
            for line in clean_rows:
                line = line.split(',')
                blocks.append(line) # Agrego estos 'bloques' a una lista general llamada "blocks" para que las conserve

            all_info = [] # Creo una nueva lista para guardar toda la información extraida de los bloque

            # Entonces, una vez generado estos bloques, que podrian ser como diccionarios, hago de nuevo otro split() a partir
            # de los dos puntos (':'). Con esto obtengo una lista con varios elementos, en la cual, en cada elemento
            # tendre en la posicion [0] las claves y en la posición [1] los valores
            for block in blocks:
                for feature in block:
                    feature = feature.lstrip().split(':')
                    all_info.append(feature)

            # Ahora que ya tengo toda a info bien extraida de los diccionarios, empiezo con la ingesta de los datos
            # al diccionario prinicipal en el cual se va a alojar toda la información de la columna, o mejor dicho, 
            # la información comprimida que estaba en cada uno de los diccionarios anidados en formato de string.
            
            for data in all_info:
                # data[0] --> claves
                # data[1] --> valores
                if len(data) == 2:
                    if data[0] in main_dictionary:
                        main_dictionary[data[0]].append(data[1])
                        try:
                            movie_id = column_id[index]
                            main_dictionary['movie_id'].append(movie_id)
                        except KeyError:
                            movie_id = column_id[index - 1]
                            movie_id = f'{movie_id} + 1'
                            main_dictionary['movie_id'].append(movie_id)
                    else: # Si no existe la clave en el main_dictonary, que la cree y que luego inserte le valor en la clave
                        main_dictionary[data[0]] = []
                        main_dictionary[data[0]].append(data[1])
                        try:
                            movie_id = column_id[index]
                            main_dictionary['movie_id'].append(movie_id)
                        except KeyError:
                            movie_id = column_id[index - 1]
                            movie_id = f'{movie_id} + 1'
                            main_dictionary['movie_id'].append(movie_id)
                else:
                    continue

            # Como para la creación de un dataframe, todas las claves del diccionario deben tener la misma longitud de valores entre si
            # Por lo tanto, para prevenir un error y que se me caiga todo el proceso anterior:
            # Obtengo el numero maximo de valores que tiene una clave
        if main_dictionary:
            longitud_maxima = max(len(valores) for valores in main_dictionary.values())

            # En esta linea me aseguro que todos los valores tengan la misma longitud que el valor de la longitud_maxima de arriba
            
            # Acá se actualiza el main_dictionary, asegurándose de que todos los valores tengan la misma longitud 
            # mediante el relleno de valores 'None' para aquellos que son mas cortos.
            main_dictionary = {clave: valores + [None] * (longitud_maxima - len(valores)) for clave, valores in main_dictionary.items()}
        
        df = pd.DataFrame(main_dictionary)
        columns_to_delete= df.columns[df.columns != 'movie_id']
        df.dropna(subset=columns_to_delete, how='all', inplace=True)
  
        # Finalmente una vez que ya tenga el main_dictonary con la misma longitud en todas las claves,
        # creo un dataframe para alojar todos los datos y luego hago que la función retorne el df
    return df

In [4]:
df_credits = df_credits.drop_duplicates(subset='id')

In [66]:
cast_data = desanidar_datos(df_credits.cast[:200],df_credits.id[:200])

In [67]:
cast_data.loc[(cast_data.credit_id.isnull()) & (cast_data.cast_id.isnull())]

,movie_id,cast_id,character,credit_id,gender,id,name,order,profile_path


In [ ]:
cast_data.to_csv('../clean_data/cast_dataset_clean.csv',sep=',',index=False)

In [7]:
cast_data.columns

Index(['cast_id', 'character', 'credit_id', 'gender', 'id', 'name', 'order',
       'profile_path'],
      dtype='object')

In [198]:
cast = cast_data.copy()
cast.drop(columns={'from Notater om kærligheden 1989 featured in segment Obstruction #4 - The Perfect Human',
                   'Lewis Carroll'}, inplace=True)

In [267]:
for column in cast:
    cast[column] = cast[column].str.lstrip()
cast.to_csv('../clean_data/cast_data.csv', index=False, lineterminator='\n')

In [24]:
crew_data = desanidar_datos(df_credits.crew,df_credits.id)

In [262]:
crew_data.head()

,credit_id,department,gender,id,job,name,profile_path
0,52fe4284c3a36847f8024f49,Directing,2,7879,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,52fe4284c3a36847f8024f4f,Writing,2,12891,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,52fe4284c3a36847f8024f55,Writing,2,7,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,52fe4284c3a36847f8024f5b,Writing,2,12892,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,52fe4284c3a36847f8024f61,Writing,0,12893,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg


In [266]:
for column in crew_data:
    crew_data[column] = crew_data[column].str.lstrip()
crew_data.to_csv('../clean_data/crew_data.csv', index=False, lineterminator='\n')

In [26]:
columns_to_clean = df_movies[['belongs_to_collection','genres','spoken_languages','production_companies','production_countries']]

In [233]:
genres_data = desanidar_datos(columns_to_clean.genres)

In [234]:
genres_data.id = genres_data.id.str.lstrip()
genres_data.name = genres_data.name.str.lstrip()

In [235]:
genres_data

,id,name
0,16,Animation
1,35,Comedy
2,10751,Family
3,12,Adventure
4,14,Fantasy
...,...,...
91101,10751,Family
91102,18,Drama
91103,28,Action
91104,18,Drama


In [21]:
def export_to_csv(dataframe, path):
    dataframe.to_csv(path, sep=',',index=False,lineterminator='\n')
    print('CSV Downloaded')

In [236]:
export_to_csv(genres_data,'../clean_data/genres_data.csv')

CSV Downloaded


In [258]:
belongs_to_collection_column = columns_to_clean.belongs_to_collection
belongs_to_collection_data = desanidar_datos(belongs_to_collection_column)
belongs_to_collection_data

,id,name,poster_path,backdrop_path
0,10194,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,119050,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
2,96871,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg
3,645,James Bond Collection,/HORpg5CSkmeQlAolx3bKMrKgfi.jpg,/6VcVl48kNKvdXOZfJPdarlUGOsk.jpg
4,117693,Balto Collection,/w0ZgH6Lgxt2bQYnf1ss74UvYftm.jpg,/9VM5LiJV0bGb1st1KyHA3cVnO2G.jpg
...,...,...,...,...
4486,37261,The Carry On Collection,/2P0HNrYgKDvirV8RCdT1rBSJdbJ.jpg,/38tF1LJN7ULeZAuAfP7beaPMfcl.jpg
4487,37261,The Carry On Collection,/2P0HNrYgKDvirV8RCdT1rBSJdbJ.jpg,/38tF1LJN7ULeZAuAfP7beaPMfcl.jpg
4488,37261,The Carry On Collection,/2P0HNrYgKDvirV8RCdT1rBSJdbJ.jpg,/38tF1LJN7ULeZAuAfP7beaPMfcl.jpg
4489,477208,DC Super Hero Girls Collection,None,None


In [259]:
belongs_to_collection_data.id = belongs_to_collection_data.id.str.lstrip()
belongs_to_collection_data.name = belongs_to_collection_data.name.str.lstrip()
belongs_to_collection_data.poster_path = belongs_to_collection_data.poster_path.str.lstrip()
belongs_to_collection_data.backdrop_path = belongs_to_collection_data.backdrop_path.str.lstrip()
export_to_csv(belongs_to_collection_data,'../clean_data/belogns_to_collection_data.csv')

CSV Downloaded


In [254]:
spoken_languages_column = columns_to_clean.spoken_languages
spoken_languages_data = desanidar_datos(spoken_languages_column)

In [255]:
spoken_languages_data

,iso_639_1,name
0,en,English
1,en,English
2,fr,Français
3,en,English
4,en,English
...,...,...
53295,en,English
53296,fa,فارسی
53297,tl,
53298,en,English


In [257]:
spoken_languages_data.iso_639_1 = spoken_languages_data.iso_639_1.str.lstrip()
spoken_languages_data.name = spoken_languages_data.name.str.lstrip()

export_to_csv(spoken_languages_data,'../clean_data/spoken_languages_data.csv')

CSV Downloaded


In [16]:
production_companies_column = columns_to_clean.production_companies
production_companies_data = desanidar_datos(production_companies_column)
production_companies_data.id = production_companies_data.id.str.lstrip()
production_companies_data.name = production_companies_data.name.str.lstrip()

In [17]:
production_countries_column = columns_to_clean.production_countries
production_countries_data = desanidar_datos(production_countries_column)
production_countries_data.iso_3166_1 = production_countries_data.iso_3166_1.str.lstrip()
production_countries_data.name = production_countries_data.name.str.lstrip()

In [19]:
production_companies_data.head()

,name,id
0,Pixar Animation Studios,3
1,TriStar Pictures,559
2,Teitler Film,2550
3,Interscope Communications,10201
4,Warner Bros.,6194


In [34]:
export_to_csv(production_companies_data,'../clean_data/production_companies_data.csv')

CSV Downloaded


In [35]:
export_to_csv(production_countries_data,'../clean_data/production_countries_data.csv')

CSV Downloaded
